# Lab 9.8: About Batch Normalization with MNIST Classifier

Edited By Steve Ive

Here, we are going to adapt the Batch Normalization to the MNIST Classifier, and compare the capability with non-adapting model.

Plus, We are going to compare various models, comparing with adapt / non-adapt models between dropout, batchnorm, weight initialization(Xavier Uniform).

- Model with Batch Normalization, Dropout, Weight Initialization
- Model With Batch Normalization, Dropout
- Model with Batch Normalization
- Model with Batch Normalization, Weight Initialization
- Model with Dropout, Weight Initiailization
- Model with Dropout
- Model with Weight Initialization.
- Model with Nothing

Furthermore, we are going to plot the accuracy of the models with matplotlib.

Reference from

https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-09_6_mnist_batchnorm.ipynb

## Imports

In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random

import matplotlib.pylab as plt

In [45]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)
random.seed(1)

if device == 'cuda':
    torch.cuda.manual_seed_all(1)

In [46]:
learning_rate = 0.01
training_epochs = 17
batch_size = 32

In [47]:
mnist_train = datasets.MNIST(root = 'MNIST_data/',
                             download = True,
                             transform = transforms.ToTensor(),
                             train = True)
mnist_test = datasets.MNIST(root = 'MNIST_data/',
                            download = True,
                            transform = transforms.ToTensor(),
                            train = False)

In [48]:
train_loader = torch.utils.data.DataLoader(dataset = mnist_train, shuffle = True, batch_size = batch_size, drop_last = True)
test_loader = torch.utils.data.DataLoader(dataset = mnist_test, shuffle = True, batch_size = batch_size, drop_last = True)

In [49]:
class Various_MNIST_Model(nn.Module):
    def __init__(self, dropout = False, batchnorm = False, weightinit = False):
        super().__init__()
        self.dropout = dropout
        self.batchnorm = batchnorm
        self.weightinit = weightinit
        self.sq = None
        self.model()

    def moduler(self, in_size, out_size, dropout, batchnorm):
        sq = ()
        sq = sq + (nn.Linear(in_size, out_size), ) #if not write "(,)" and just writing () makes error occur since not tuple
        if batchnorm:
            sq = sq + (nn.BatchNorm1d(out_size), )
        sq = sq + (nn.ReLU(), )
        if dropout:
            sq = sq + (nn.Dropout(p = 0.3), )
        return sq
        
    def model(self):
        self.sq = nn.Sequential(
            *self.moduler(784, 32, self.dropout, self.batchnorm),
            *self.moduler(32, 32, self.dropout, self.batchnorm),
            *self.moduler(32, 32, self.dropout, self.batchnorm),
            nn.Linear(32, 10)
        )
        print(self.sq)
        if self.weightinit:
            self.weightinitializer()

    def weightinitializer(self):
        for layer in self.sq:
            if type(layer) == nn.Linear:
                torch.nn.init.xavier_uniform_(layer.weight)

    def forward(self, x):
        return self.sq(x)
    

In [54]:
print('Dropout + Batchnorm + weightinit')
model_dropout_batchnorm_weightinit = Various_MNIST_Model(dropout = True, batchnorm = True, weightinit = True)
optim_dbw = optim.Adam(model_dropout_batchnorm_weightinit.parameters(), lr = learning_rate)
print('Dropout + Batchnorm')
model_dropout_batchnorm = Various_MNIST_Model(dropout = True, batchnorm = True)
optim_db = optim.Adam(model_dropout_batchnorm.parameters(), lr= learning_rate)
print('Dropout')
model_dropout = Various_MNIST_Model(dropout = True)
optim_d = optim.Adam(model_dropout.parameters(), lr= learning_rate)
print('Batchnorm')
model_batchnorm = Various_MNIST_Model(batchnorm = True)
optim_b = optim.Adam(model_batchnorm.parameters(), lr=learning_rate)
print('Batchnorm + weightinit')
model_batchnorm_weightinit = Various_MNIST_Model(batchnorm=True, weightinit = True)
optim_bw = optim.Adam(model_batchnorm_weightinit.parameters(), lr=learning_rate)
print('Dropout + weightinit')
model_dropout_weightinit = Various_MNIST_Model(dropout = True, weightinit = True)
optim_dw = optim.Adam(model_dropout_weightinit.parameters(), lr = learning_rate)
print('WeightInit')
model_weightinit = Various_MNIST_Model(weightinit = True)
optim_w = optim.Adam(model_weightinit.parameters(), lr= learning_rate)
print('Nothing')
model_nothing = Various_MNIST_Model()
optim = optim.Adam(model_nothing.parameters(), lr = learning_rate)

Dropout + Batchnorm + weightinit
Sequential(
  (0): Linear(in_features=784, out_features=32, bias=True)
  (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.3, inplace=False)
  (4): Linear(in_features=32, out_features=32, bias=True)
  (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): Dropout(p=0.3, inplace=False)
  (8): Linear(in_features=32, out_features=32, bias=True)
  (9): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): Dropout(p=0.3, inplace=False)
  (12): Linear(in_features=32, out_features=10, bias=True)
)
Dropout + Batchnorm
Sequential(
  (0): Linear(in_features=784, out_features=32, bias=True)
  (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.3, inplace=False)
  (4): Linear(in_features=32, out_features=32, bias=True)
  (5): BatchNorm

## Train and save loss and accuracy every epoch

In [ ]:
train_losses = []
train_accs = []

valid_losses = []
train_accs = []

train_total_batch = len(train_loader)
test_tot